In [6]:
import torch
import time

# Define the function to perform the computation
def compute_payoff(s0, r, sigma, dt, K, num_simulations, z):
    # Compute the stock price at time t
    s = s0 * torch.exp((r - sigma**torch.tensor(2.0) / torch.tensor(2.0)) * dt + sigma * torch.sqrt(dt) * z)

    # Calculate the payoff
    payoff = torch.where(s > K, s - K, torch.tensor(0.0))

    # Compute the average payoff
    average_payoff = torch.exp(-r * dt) * payoff.sum() / num_simulations

    return average_payoff

# Compile the function with torch.jit.script
scripted_compute_payoff = torch.jit.script(compute_payoff)

# Function to measure the runtime and compute payoff
def run_simulation(s0, r, sigma, dt, K, num_simulations, z):
    # Convert inputs to tensors
    s0_tensor = torch.tensor(s0, requires_grad=True)
    r_tensor = torch.tensor(r, requires_grad=True)
    sigma_tensor = torch.tensor(sigma, requires_grad=True)
    dt_tensor = torch.tensor(dt, requires_grad=True)
    K_tensor = torch.tensor(K, requires_grad=True)
    num_simulations_tensor = torch.tensor(num_simulations)

    # Call the scripted function
    average_payoff = scripted_compute_payoff(s0_tensor, r_tensor, sigma_tensor, dt_tensor, K_tensor, num_simulations_tensor, z)

    # Perform backward pass
    average_payoff.backward()

    # value_standard = average_payoff.item()
    # delta_standard = s0_tensor.grad.item()

    return average_payoff.item(), s0_tensor.grad.item()

torch.manual_seed(1)

# Function to measure the runtime and compute payoff without JIT
def run_simulation_non_jit(s0, r, sigma, dt, K, num_simulations, z):
    # Convert inputs to tensors
    s0_tensor = torch.tensor(s0, requires_grad=True)
    r_tensor = torch.tensor(r, requires_grad=True)
    sigma_tensor = torch.tensor(sigma, requires_grad=True)
    dt_tensor = torch.tensor(dt, requires_grad=True)
    K_tensor = torch.tensor(K, requires_grad=True)

    # Call the scripted function
    average_payoff = compute_payoff(s0_tensor, r_tensor, sigma_tensor, dt_tensor, K_tensor, num_simulations, z)

    s0_tensor.grad = None
    # Perform backward pass
    average_payoff.backward()

    # value_standard = average_payoff.item()
    # delta_standard = s0_tensor.grad.item()

    return average_payoff.item(), s0_tensor.grad.item()


# Function to run the simulation x times and average the runtime
def run_multiple_simulations(s0, r, sigma, dt, B, num_simulations, x):
    total_time_jit = 0.0
    total_time_non_jit = 0.0

    initial_s0 = s0

    torch.manual_seed(1)
    z = torch.normal(mean=0, std=1, size=(1,num_simulations))

    results = []
    deltas = []

    results_jit = []
    deltas_jit =[]

    for i in range(x):
        # JIT version
        
        current_s0 = initial_s0 #+ 0.0001 * i
        start_time = time.time()
        average_payoff_jit, delta_jit = run_simulation(current_s0, r, sigma, dt, B, num_simulations, z)
        end_time = time.time()
        total_time_jit += (end_time - start_time)

        # Non-JIT version
        start_time = time.time()

        average_payoff_non_jit, delta_non_jit = run_simulation_non_jit(current_s0, r, sigma, dt, B, num_simulations, z)

        end_time = time.time()
        total_time_non_jit += (end_time - start_time)

        results.append(average_payoff_non_jit)
        deltas.append(delta_non_jit)

        results_jit.append(average_payoff_jit)
        deltas_jit.append(delta_jit)

    average_runtime_jit = total_time_jit / x
    average_runtime_non_jit = total_time_non_jit / x

    mean_results = sum(results) / x
    mean_deltas = sum(deltas) / x

    mean_results_jit = sum(results_jit) /x
    mean_deltas_jit = sum(deltas_jit) / x

    return mean_results_jit, mean_deltas_jit,  average_runtime_jit, mean_results, mean_deltas, average_runtime_non_jit, 


# Initial parameters
s0 = 100.0
r = 0.05
sigma = 0.3
dt = 1.0
K = 110.0
num_simulations = 1000000
x = 100  # Number of times to run the simulation


# Run the simulations and average the runtime
average_payoff_jit, average_delta_jit, average_runtime_jit, average_payoff_non_jit, average_delta_non_jit, average_runtime_non_jit = run_multiple_simulations(s0, r, sigma, dt, K, num_simulations, x)

# Print the average payoff and average runtime for both JIT and non-JIT
# print(f"JIT - Average payoff: {average_payoff_jit.item()}")
# print(f"JIT - Average runtime over {x} runs: {average_runtime_jit} seconds")

# print(f"Non-JIT - Average payoff: {average_payoff_non_jit.item()}")
# print(f"Non-JIT - Average runtime over {x} runs: {average_runtime_non_jit} seconds")

# Output results in table format
print("{:<20} {:<12} {:<12} {:<15} ".format('Backend', 'Result', 'Gradient', 'mean runtime'))
print("{:<20} {:<12.6f} {:<12.6f} {:<15.6f} ".format("torch_native_jit", average_payoff_jit, average_delta_jit, average_runtime_jit))
print("{:<20} {:<12.6f} {:<12.6f} {:<15.6f} ".format("torch_native", average_payoff_non_jit , average_delta_non_jit,  average_runtime_non_jit))


Backend              Result       Gradient     mean runtime    
torch_native_jit     10.020878    0.499224     0.001674        
torch_native         10.020878    0.499224     0.001456        


In [7]:
import os
import sys

os.getcwd()  # Check the current working directory
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..\..')))

import DifferentiationInterface.src.diffipy.diffipy as dp

In [8]:
import time
import torch

# Initialize variables and constants
s0_value = 100.0
K_value = 110.0
r_value = 0.05
sigma_value = 0.3
dt_value = 1.0
z_value = 0.5

N = 1000000
seed = 1

# Computation settings
dp.setBackend('torch')

# Define variables
s0 = dp.variable(s0_value, 'input','s0')
K = dp.variable(K_value, 'input','K')
r = dp.variable(r_value, 'input','r')
sigma = dp.variable(sigma_value, 'input','sigma')
dt = dp.variable(dt_value, 'input','dt')
z = dp.variable(z_value, 'randomVariableNormal','z')

# Record Tape: Standard Monte Carlo

s = s0 * dp.exp((r - sigma **2 / 2) * dt + sigma * dp.sqrt(dt) * z)
payoff =  dp.if_(s > K, s - K, 0)
PV_standard = dp.exp(-r * dt) * dp.sum(payoff) / N

test_iterations = 100
warumup_iterations = 10

###
### Test performance of graph eval
###

dp.seed(seed)

total_time = 0.0
results_standard = []
deltas_standard = []
times = []

z_precomputed = torch.normal(mean=0, std=1, size=(1, N))

for _ in range(test_iterations):
    tic = time.time()

    z.value = z_precomputed
    result_standard = PV_standard.eval()
    delta_standard = PV_standard.grad(s0)

    toc = time.time()
    spent = toc - tic
    times.append(spent)
    total_time += spent
    results_standard.append(result_standard)
    deltas_standard.append(delta_standard)

###
### Test performance of optimized executable
###

myfunc = PV_standard.get_optimized_executable()

dp.seed(seed)
time_total_optimized = 0
times_optimized = []
results_optimized = []
deltas_optimized = []

z_precomputed = torch.normal(mean=0, std=1, size=(1, N))

s0_tensor = torch.tensor(s0_value, requires_grad=True)
r_tensor = torch.tensor(r_value, requires_grad=True)
sigma_tensor = torch.tensor(sigma_value, requires_grad=True)
dt_tensor = torch.tensor(dt_value, requires_grad=True)
K_tensor = torch.tensor(K_value, requires_grad=True)

#Warumup
for _ in range(warumup_iterations):
    result_optimized = myfunc(s0=s0_tensor, K=K_tensor, r=r_tensor, sigma=sigma_tensor, dt = dt_tensor, z=z_precomputed)
    s0_tensor.grad = None
    result_optimized.backward()
    derivative_optimized = s0_tensor.grad.item()


for _ in range(test_iterations):
    tic = time.time()

    result_optimized = myfunc(s0=s0_tensor, K=K_tensor, r=r_tensor, sigma=sigma_tensor, dt = dt_tensor, z=z_precomputed)
    s0_tensor.grad = None
    result_optimized.backward()
    derivative_optimized = s0_tensor.grad.item()

    toc = time.time()
    spent = toc - tic
    times_optimized.append(spent)
    time_total_optimized += spent

    results_optimized.append(result_optimized.item())
    deltas_optimized.append(derivative_optimized)
    
# Compute runtimes
mean_time_standard =  total_time / test_iterations
variance_time_standard =  sum((time - mean_time_standard) ** 2 for time in times) / (test_iterations - 1)    
mean_time_optimized = time_total_optimized / test_iterations
variance_time_optimized = sum((time - mean_time_optimized) ** 2 for time in times_optimized) / (test_iterations - 1)

# Output results in table format
print("{:<20} {:<12} {:<12} {:<15} {:<16}".format('Backend', 'Result', 'Gradient', 'mean runtime', 'variance runtime'))
print("{:<20} {:<12.6f} {:<12.6f} {:<15.6f} {:<15.6f}".format("torch_dipy", sum(results_standard) / test_iterations, sum(deltas_standard) / test_iterations, mean_time_standard, variance_time_standard))
print("{:<20} {:<12.6f} {:<12.6f} {:<15.6f} {:<15.6f}".format("torch_dipy_optimized", sum(results_optimized) / test_iterations, sum(deltas_optimized) / test_iterations, mean_time_optimized, variance_time_optimized))


Backend              Result       Gradient     mean runtime    variance runtime
torch_dipy           10.020878    0.499224     0.002480        0.000022       
torch_dipy_optimized 10.020878    0.499224     0.001522        0.000014       
